In [10]:
import os
import pandas as pd
# Show all columns
pd.set_option('display.max_columns', None)
import warnings

warnings.filterwarnings("ignore")

In [2]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Download image data using provided download CLI
Below we download all metadata and resized images (500p) for a FungiTastic-Mini subset.

In [3]:
# !python ../../dataset/download.py --metadata --images --subset "m" --size "500" --save_path "./"

In [4]:
!ls ./FungiTastic/metadata/FungiTastic-Mini/

FungiTastic-Mini-ClosedSet-Test.csv FungiTastic-Mini-OpenSet-Test.csv
FungiTastic-Mini-ClosedSet-Val.csv  FungiTastic-Mini-OpenSet-Val.csv
FungiTastic-Mini-DNA-Test.csv       FungiTastic-Mini-Train.csv


In [5]:
!ls ./FungiTastic/FungiTastic-Mini

dna-test test     train    val


## Load and preprocess metadata

In [6]:
print(os.getcwd())

/Users/jeremycui/Documents/UCB_MIDS/DATASCI207/fungitastic-classification-datasci207-Fall-2025/baselines/closed_set


In [7]:
BASE_METADATA_PATH = f"{os.getcwd()}/FungiTastic/metadata/FungiTastic-Mini/"

train_df = pd.read_csv(f"{BASE_METADATA_PATH}FungiTastic-Mini-Train.csv")
val_df = pd.read_csv(f"{BASE_METADATA_PATH}FungiTastic-Mini-ClosedSet-Val.csv")
test_df = pd.read_csv(f"{BASE_METADATA_PATH}FungiTastic-Mini-ClosedSet-Test.csv")

In [11]:
train_df

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,hasCoordinate,species,iucnRedListCategory,substrate,latitude,longitude,coorUncert,observationID,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,image_path
0,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,3032614317,Sjælland,Næstved,0-3032614317.JPG,119,wood,0,35.0,5.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-30...
1,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,3032614317,Sjælland,Næstved,1-3032614317.JPG,119,wood,0,35.0,5.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-30...
2,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,0-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-30...
3,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,1-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-30...
4,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,2-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/2-30...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46837,2021-09-29,2021,9.0,29.0,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,rosea,True,Mycena rosea,NE,soil,54.922850,36.346589,15.0,3429074356,Kaluga,Maloyaroslavetskiy rayon,1-3429074356.JPG,109,jord,1,NaN,NaN,boreal,./FungiTastic/FungiTastic-Mini/train/500p/1-34...
46838,2021-09-29,2021,9.0,29.0,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,rosea,True,Mycena rosea,NE,soil,54.922850,36.346589,15.0,3429074356,Kaluga,Maloyaroslavetskiy rayon,2-3429074356.JPG,109,jord,1,NaN,NaN,boreal,./FungiTastic/FungiTastic-Mini/train/500p/2-34...
46839,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,4100099773,Hovedstaden,Halsnæs,0-4100099773.JPG,179,jord,0,0.0,17.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-41...
46840,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,4100099773,Hovedstaden,Halsnæs,1-4100099773.JPG,179,jord,0,0.0,17.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-41...


### Setting image_path to the images.

In [37]:
BASE_IMAGE_PATH = "./FungiTastic/FungiTastic-Mini"
SIZE = "500p"

train_df["image_path"] = train_df.filename.apply(lambda filename: f"{BASE_IMAGE_PATH}/train/{SIZE}/{filename}")
val_df["image_path"] = val_df.filename.apply(lambda filename: f"{BASE_IMAGE_PATH}/val/{SIZE}/{filename}")

In [38]:
train_df

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,hasCoordinate,species,iucnRedListCategory,substrate,latitude,longitude,coorUncert,observationID,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,image_path
0,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,3032614317,Sjælland,Næstved,0-3032614317.JPG,119,wood,0,35.0,5.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-30...
1,2021-02-01,2021,2.0,1.0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,3032614317,Sjælland,Næstved,1-3032614317.JPG,119,wood,0,35.0,5.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-30...
2,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,0-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-30...
3,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,1-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-30...
4,2008-09-01,2008,9.0,1.0,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,3036761318,Midtjylland,Århus,2-3036761318.JPG,144,jord,0,6.0,10.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/2-30...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46837,2021-09-29,2021,9.0,29.0,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,rosea,True,Mycena rosea,NE,soil,54.922850,36.346589,15.0,3429074356,Kaluga,Maloyaroslavetskiy rayon,1-3429074356.JPG,109,jord,1,NaN,NaN,boreal,./FungiTastic/FungiTastic-Mini/train/500p/1-34...
46838,2021-09-29,2021,9.0,29.0,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,rosea,True,Mycena rosea,NE,soil,54.922850,36.346589,15.0,3429074356,Kaluga,Maloyaroslavetskiy rayon,2-3429074356.JPG,109,jord,1,NaN,NaN,boreal,./FungiTastic/FungiTastic-Mini/train/500p/2-34...
46839,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,4100099773,Hovedstaden,Halsnæs,0-4100099773.JPG,179,jord,0,0.0,17.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/0-41...
46840,2021-09-27,2021,9.0,27.0,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,4100099773,Hovedstaden,Halsnæs,1-4100099773.JPG,179,jord,0,0.0,17.0,continental,./FungiTastic/FungiTastic-Mini/train/500p/1-41...


In [39]:
train_df.columns

Index(['eventDate', 'year', 'month', 'day', 'habitat', 'countryCode',
       'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
       'genus', 'specificEpithet', 'hasCoordinate', 'species',
       'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
       'coorUncert', 'observationID', 'region', 'district', 'filename',
       'category_id', 'metaSubstrate', 'poisonous', 'elevation', 'landcover',
       'biogeographicalRegion', 'image_path'],
      dtype='object')

### Preprocess Metadata

In [42]:
metadata_feat_list = [
    'habitat', 'countryCode',
    'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
    'genus', 'specificEpithet', 'hasCoordinate', 'species',
    'iucnRedListCategory', 'substrate', 'latitude', 'longitude',
    'coorUncert', 'region', 'district',
    'metaSubstrate', 'poisonous', 'elevation', 'landcover',
    'biogeographicalRegion'
]
train_df_metadata = train_df[metadata_feat_list]

In [43]:
train_df_metadata.describe()

,latitude,longitude,coorUncert,poisonous,elevation,landcover
count,46841.000000,46841.000000,46123.000000,46842.000000,46799.000000,46799.000000
mean,55.772692,11.051221,71.644971,0.124738,67.640377,11.569093
std,1.037340,2.374950,329.375651,0.330426,139.218109,5.490019
min,-37.423181,-123.014002,1.000000,0.000000,0.000000,1.000000
25%,55.387692,9.825039,10.000000,0.000000,0.000000,8.000000
50%,55.786520,11.372933,25.000000,0.000000,13.000000,12.000000
75%,56.090559,12.292067,50.000000,0.000000,73.000000,17.000000
max,69.727863,144.555467,24069.000000,1.000000,2313.000000,17.000000


### Frequency Encoding

In [44]:
cat_metadata_feat_list = [
    'habitat', 'countryCode',
    'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family',
    'genus', 'specificEpithet', 'hasCoordinate', 'species',
    'iucnRedListCategory', 'substrate', 'coorUncert', 'region', 'district', 
    'metaSubstrate', 'biogeographicalRegion'
]

cat_train_df_metadata = train_df_metadata[cat_metadata_feat_list]

In [48]:
cat_vc_dict = {}

for cat in list(cat_train_df_metadata.columns):
    # cat_vc = cat_train_df_metadata[cat].value_counts()
    cat_vc_dict[cat] = dict(cat_train_df_metadata[cat].value_counts())

In [49]:
cat_vc_dict

{'habitat': {'Deciduous woodland': np.int64(11687),
  'Mixed woodland (with coniferous and deciduous trees)': np.int64(8470),
  'coniferous woodland/plantation': np.int64(6249),
  'Unmanaged deciduous woodland': np.int64(5026),
  'park/churchyard': np.int64(3462),
  'Unmanaged coniferous woodland': np.int64(1667),
  'natural grassland': np.int64(1586),
  'lawn': np.int64(1377),
  'roadside': np.int64(1143),
  'garden': np.int64(1074),
  'Thorny scrubland': np.int64(1027),
  'Bog woodland': np.int64(545),
  'Forest bog': np.int64(541),
  'hedgerow': np.int64(419),
  'dune': np.int64(370),
  'wooded meadow, grazing forest': np.int64(368),
  'bog': np.int64(296),
  'Willow scrubland': np.int64(292),
  'heath': np.int64(289),
  'salt meadow': np.int64(235),
  'other habitat': np.int64(161),
  'Acidic oak woodland': np.int64(159),
  'meadow': np.int64(145),
  'gravel or clay pit': np.int64(96),
  'improved grassland': np.int64(69),
  'fallow field': np.int64(34),
  'ditch': np.int64(26),
  

In [52]:
train_df_metadata_cat_enc = train_df_metadata.copy()

In [53]:
for col, mapping in cat_vc_dict.items():
    # Use .map with dict (fast, vectorized, memory-friendly)
    train_df_metadata_cat_enc[col] = train_df_metadata[col].map(mapping)

In [55]:
train_df_metadata

,habitat,countryCode,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,hasCoordinate,species,iucnRedListCategory,substrate,latitude,longitude,coorUncert,region,district,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,habitat_freq,countryCode_freq,scientificName_freq,kingdom_freq,phylum_freq,class_freq,order_freq,family_freq,genus_freq,specificEpithet_freq,hasCoordinate_freq,species_freq,iucnRedListCategory_freq,substrate_freq,coorUncert_freq,region_freq,district_freq,metaSubstrate_freq,biogeographicalRegion_freq
0,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,Sjælland,Næstved,wood,0,35.0,5.0,continental,8470,46242,568,46842,46842,46842,29197,13155,13155,568,46841,568,41073,4676.0,7221.0,8846.0,1169.0,7385,41592.0
1,Mixed woodland (with coniferous and deciduous ...,DK,Mycena tintinnabulum (Paulet) Quél.,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,tintinnabulum,True,Mycena tintinnabulum,NE,dead wood (including bark),55.280440,11.761377,15.0,Sjælland,Næstved,wood,0,35.0,5.0,continental,8470,46242,568,46842,46842,46842,29197,13155,13155,568,46841,568,41073,4676.0,7221.0,8846.0,1169.0,7385,41592.0
2,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,Midtjylland,Århus,jord,0,6.0,10.0,continental,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,5927.0,9307.0,1753.0,34454,41592.0
3,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,Midtjylland,Århus,jord,0,6.0,10.0,continental,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,5927.0,9307.0,1753.0,34454,41592.0
4,Deciduous woodland,DK,Russula cyanoxantha (Schaeff.) Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,cyanoxantha,True,Russula cyanoxantha,NE,soil,56.249168,10.169204,75.0,Midtjylland,Århus,jord,0,6.0,10.0,continental,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,5927.0,9307.0,1753.0,34454,41592.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46837,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,rosea,True,Mycena rosea,NE,soil,54.922850,36.346589,15.0,Kaluga,Maloyaroslavetskiy rayon,jord,1,NaN,NaN,boreal,5026,28,874,46842,46842,46842,29197,13155,13155,874,46841,874,41073,34454.0,7221.0,26.0,26.0,34454,210.0
46838,Unmanaged deciduous woodland,RU,Mycena rosea Gramberg,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Mycenaceae,Mycena,rosea,True,Mycena rosea,NE,soil,54.922850,36.346589,15.0,Kaluga,Maloyaroslavetskiy rayon,jord,1,NaN,NaN,boreal,5026,28,874,46842,46842,46842,29197,13155,13155,874,46841,874,41073,34454.0,7221.0,26.0,26.0,34454,210.0
46839,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,Hovedstaden,Halsnæs,jord,0,0.0,17.0,continental,11687,46242,1018,46842,46842,46842,15585,15585,15585,1018,46841,1018,41073,34454.0,247.0,14524.0,856.0,34454,41592.0
46840,Deciduous woodland,DK,Russula ochroleuca Fr.,Fungi,Basidiomycota,Agaricomycetes,Russulales,Russulaceae,Russula,ochroleuca,True,Russula ochroleuca,NE,soil,55.954525,11.907830,6.0,Hovedstaden,Halsnæs,jord,0,0.0,17.0,continental,11687,46242,1018,46842,46842,46842,15585,15585,15585,1018,46841,1018,41073,34454.0,247.0,14524.0,856.0,34454,41592.0


In [54]:
train_df_metadata_cat_enc

,habitat,countryCode,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,hasCoordinate,species,iucnRedListCategory,substrate,latitude,longitude,coorUncert,region,district,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,habitat_freq,countryCode_freq,scientificName_freq,kingdom_freq,phylum_freq,class_freq,order_freq,family_freq,genus_freq,specificEpithet_freq,hasCoordinate_freq,species_freq,iucnRedListCategory_freq,substrate_freq,coorUncert_freq,region_freq,district_freq,metaSubstrate_freq,biogeographicalRegion_freq
0,8470,46242,568,46842,46842,46842,29197,13155,13155,568,46841,568,41073,4676.0,55.280440,11.761377,7221.0,8846.0,1169.0,7385,0,35.0,5.0,41592.0,8470,46242,568,46842,46842,46842,29197,13155,13155,568,46841,568,41073,4676.0,7221.0,8846.0,1169.0,7385,41592.0
1,8470,46242,568,46842,46842,46842,29197,13155,13155,568,46841,568,41073,4676.0,55.280440,11.761377,7221.0,8846.0,1169.0,7385,0,35.0,5.0,41592.0,8470,46242,568,46842,46842,46842,29197,13155,13155,568,46841,568,41073,4676.0,7221.0,8846.0,1169.0,7385,41592.0
2,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,56.249168,10.169204,5927.0,9307.0,1753.0,34454,0,6.0,10.0,41592.0,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,5927.0,9307.0,1753.0,34454,41592.0
3,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,56.249168,10.169204,5927.0,9307.0,1753.0,34454,0,6.0,10.0,41592.0,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,5927.0,9307.0,1753.0,34454,41592.0
4,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,56.249168,10.169204,5927.0,9307.0,1753.0,34454,0,6.0,10.0,41592.0,11687,46242,678,46842,46842,46842,15585,15585,15585,678,46841,678,41073,34454.0,5927.0,9307.0,1753.0,34454,41592.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46837,5026,28,874,46842,46842,46842,29197,13155,13155,874,46841,874,41073,34454.0,54.922850,36.346589,7221.0,26.0,26.0,34454,1,NaN,NaN,210.0,5026,28,874,46842,46842,46842,29197,13155,13155,874,46841,874,41073,34454.0,7221.0,26.0,26.0,34454,210.0
46838,5026,28,874,46842,46842,46842,29197,13155,13155,874,46841,874,41073,34454.0,54.922850,36.346589,7221.0,26.0,26.0,34454,1,NaN,NaN,210.0,5026,28,874,46842,46842,46842,29197,13155,13155,874,46841,874,41073,34454.0,7221.0,26.0,26.0,34454,210.0
46839,11687,46242,1018,46842,46842,46842,15585,15585,15585,1018,46841,1018,41073,34454.0,55.954525,11.907830,247.0,14524.0,856.0,34454,0,0.0,17.0,41592.0,11687,46242,1018,46842,46842,46842,15585,15585,15585,1018,46841,1018,41073,34454.0,247.0,14524.0,856.0,34454,41592.0
46840,11687,46242,1018,46842,46842,46842,15585,15585,15585,1018,46841,1018,41073,34454.0,55.954525,11.907830,247.0,14524.0,856.0,34454,0,0.0,17.0,41592.0,11687,46242,1018,46842,46842,46842,15585,15585,15585,1018,46841,1018,41073,34454.0,247.0,14524.0,856.0,34454,41592.0


### Preprocess Images

### Update the column name to fit the FGVC expected naming convention and save the updated metadata

In [10]:
!mkdir metadata

train_df = train_df.rename(columns={"category_id":"class_id"})
val_df = val_df.rename(columns={"category_id":"class_id"})

train_df.to_csv("./metadata/FungiTastic-Mini-Train.csv", index=None)
val_df.to_csv("./metadata/FungiTastic-Mini-ClosedSet-Val.csv", index=None)

mkdir: metadata: File exists


## Train single model with a config file.

### Environment variables
Select which GPU to use and add your WANDB_ENTITY and WANDB_PROJECT you want to log into etc.

In [11]:
%env CUDA_DEVICES = 0
%env WANDB_ENTITY = zcu_cv
%env HFHUB_OWNER = BVRA
%env WANDB_PROJECT = FungiTastic
%env TRAIN_METADATA_PATH = ./metadata/FungiTastic-Mini-Train.csv
%env VAL_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Val.csv
%env TEST_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Test.csv

os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["CUDA_DEVICES"]

env: CUDA_DEVICES=0
env: WANDB_ENTITY=zcu_cv
env: HFHUB_OWNER=BVRA
env: WANDB_PROJECT=FungiTastic
env: TRAIN_METADATA_PATH=./metadata/FungiTastic-Mini-Train.csv
env: VAL_METADATA_PATH=./metadata/FungiTastic-Mini-ClosedSet-Val.csv
env: TEST_METADATA_PATH=./metadata/FungiTastic-Mini-ClosedSet-Test.csv


In [14]:
!which python
!python -m pip install wandb

/Users/jeremycui/Documents/UCB_MIDS/DATASCI207/fungitastic-classification-datasci207-Fall-2025/.dataset_demo_venv/bin/python
  Using cached wandb-0.22.0-py3-none-macosx_12_0_arm64.whl.metadata (10 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached wandb-0.22.0-py3-none-macosx_12_0_arm64.whl (18.3 MB)
Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl (426 kB)
Using cached click-8.3.0-py3-none-any.whl (107 kB)
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [wandb]32m6/7 [wandb]uf]k]

[notice] A new release of pip i

In [16]:
!python train.py \
    --train-path "./metadata/FungiTastic-Mini-Train.csv" \
    --val-path "./metadata/FungiTastic-Mini-ClosedSet-Val.csv" \
    --test-path "./metadata/FungiTastic-Mini-ClosedSet-Test.csv" \
    --config-path ./configs/default_config.yaml \
    --cuda-devices $CUDA_DEVICES \
    --wandb-entity $WANDB_ENTITY \
    --wandb-project $WANDB_PROJECT \
    --hfhub-owner $HFHUB_OWNER

(script) INFO: Loading training config.
(script) DEBUG: Extra arguments passed to the script: {}
(script) INFO: Setting run name: resnet18.a1_in1k-CrossEntropyLoss-light
(script) INFO: Using experiment directory: ../runs/resnet18.a1_in1k-CrossEntropyLoss-light/exp2
(script) INFO: Using training configuration: {
    "augmentations": "light",
    "image_size": [
        224,
        224
    ],
    "dataset": "FungiTastic-Mini",
    "architecture": "resnet18.a1_in1k",
    "train": null,
    "loss": "CrossEntropyLoss",
    "optimizer": "SGD",
    "scheduler": "plateau",
    "epochs": 1,
    "learning_rate": 0.01,
    "batch_size": 64,
    "accumulation_steps": 1,
    "random_seed": 777,
    "workers": 8,
    "multigpu": false,
    "tags": [
        "FungiTastic-M-CloseSet",
        "224x224",
        "Convolutional"
    ],
    "root_path": "../",
    "run_name": "resnet18.a1_in1k-CrossEntropyLoss-light",
    "exp_name": "exp2",
    "exp_path": "../runs/resnet18.a1_in1k-CrossEntropyLoss-lig

## Running Sweep Training

### Environment variables
Select which GPU to use and add your WANDB_ENTITY and WANDB_PROJECT you want to log into etc.

In [ ]:
%env CUDA_DEVICES = "changethis"
%env WANDB_ENTITY = "changethis"
%env HFHUB_OWNER = "changethis"
%env WANDB_PROJECT = "changethis"
%env TRAIN_METADATA_PATH = ./metadata/FungiTastic-Mini-Train.csv
%env VAL_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Val.csv
%env TEST_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Test.csv

os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["CUDA_DEVICES"]

In [ ]:
!wandb sweep ./sweep_configs/FungiTastic-M-CloseSet-224.yaml

In [ ]:
!wandb agent <WANDB_ENTITY>/<WANDB_PROJECT>/.....